In [1]:
%matplotlib 
%load_ext autoreload
%autoreload 2

Using matplotlib backend: Qt4Agg


In [2]:
import sys
sys.path.append("/mnt/Data/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [3]:
import os
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import colormaps as mycmps
import signale.tools as tools
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import scipy.signal as scsig
from scipy.stats import sem
from matplotlib import rcParams
import trajectory

/home/chenani/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/home/chenani/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Wavelet Analysis Class

In [4]:
"""
A module which implements the continuous wavelet transform

---------------------------------------------------------
Code released under the BSD 3-clause licence.

Copyright (c) 2012, R W Fearick, University of Cape Town
All rights reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the
following conditions are met:

    * Redistributions of source code must retain the above copyright notice, this list of conditions and
        the following disclaimer.
    * Redistributions in binary form must reproduce the above copyright notice, this list of conditions and
        the following disclaimer in the documentation and/or other materials provided with the distribution.
    * Neither the name of the University of Cape Town nor the names of its contributors may be used to endorse
        or promote products derived from this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, 
INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR 
SERVICES;LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
WHETHER IN CONTRACT,STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE
USE OF THIS SOFTWARE,EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
---------------------------------------------------------

Wavelet classes:
Morlet
MorletReal
MexicanHat
Paul2      : Paul order 2
Paul4      : Paul order 4
DOG1       : 1st Derivative Of Gaussian
DOG4       : 4th Derivative Of Gaussian
Haar       : Unnormalised version of continuous Haar transform
HaarW      : Normalised Haar

Usage e.g.
wavelet=Morlet(data, largestscale=2, notes=0, order=2, scaling="log")
 data:  Numeric array of data (float), with length ndata.
        Optimum length is a power of 2 (for FFT)
        Worst-case length is a prime
 largestscale:
        largest scale as inverse fraction of length
        scale = len(data)/largestscalax1.set_xticklabels([e
        smallest scale should be >= 2 for meaningful data
 notes: number of scale intervals per octave
        if notes == 0, scales are on a linear increment
 order: order of wavelet for wavelets with variable order
        [Paul, DOG, ..]
 scaling: "linear" or "log" scaling of the wavelet scale.
        Note that feature width in the scale direction
        is constant on a log scale.
        
Attributes of instance:
wavelet.cwt:       2-d array of Wavelet coefficients, (nscales,ndata)
wavelet.nscale:    Number of scale intervals
wavelet.scales:    Array of scale values
                   Note that meaning of the scale will depend on the family
wavelet.fourierwl: Factor to multiply scale by to get scale
                   of equivalent FFT
                   Using this factor, different wavelet families will
                   have comparable scales

References:
A practical guide to wavelet analysis
C Torrance and GP Compo
Bull Amer Meteor Soc Vol 79 No 1 61-78 (1998)
naming below vaguely follows this.

updates:
(24/2/07):  Fix Morlet so can get MorletReal by cutting out H
(10/04/08): Numeric -> numpy
(25/07/08): log and lin scale increment in same direction!
            swap indices in 2-d coeffiecient matrix
            explicit scaling of scale axis
"""

class Cwt:
    """
    Base class for continuous wavelet transforms
    Implements cwt via the Fourier transform
    Used by subclass which provides the method wf(self,s_omega)
    wf is the Fourier transform of the wavelet function.
    Returns an instance.
    """

    fourierwl=1.00

    def _log2(self, x):
        # utility function to return (integer) log2
        return int( np.log(float(x))/ np.log(2.0)+0.0001 )

    def __init__(self, data, largestscale=1, notes=0, order=2, scaling='linear'):
        """
        Continuous wavelet transform of data

        data:    data in array to transform, length must be power of 2
        notes:   number of scale intervals per octave
        largestscale: largest scale as inverse fraction of length
                 of data array
                 scale = len(data)/largestscale
                 smallest scale should be >= 2 for meaningful data
        order:   Order of wavelet basis function for some families
        scaling: Linear or log
        """
        ndata = len(data)
        self.order=order
        self.scale=largestscale
        self._setscales(ndata,largestscale,notes,scaling)
        self.cwt= np.zeros((self.nscale,ndata), np.complex64)
        omega= np.array(range(0,ndata/2)+range(-ndata/2,0))*(2.0*np.pi/ndata)
        datahat=np.fft.fft(data)
        self.fftdata=datahat
        #self.psihat0=self.wf(omega*self.scales[3*self.nscale/4])
        # loop over scales and compute wvelet coeffiecients at each scale
        # using the fft to do the convolution
        for scaleindex in range(self.nscale):
            currentscale=self.scales[scaleindex]
            self.currentscale=currentscale  # for internal use
            s_omega = omega*currentscale
            psihat=self.wf(s_omega)
            psihat = psihat *  np.sqrt(2.0*np.pi*currentscale)
            convhat = psihat * datahat
            W    = np.fft.ifft(convhat)
            self.cwt[scaleindex,0:ndata] = W 
        return
    
    def _setscales(self,ndata,largestscale,notes,scaling):
        """
        if notes non-zero, returns a log scale based on notes per ocave
        else a linear scale
        (25/07/08): fix notes!=0 case so smallest scale at [0]
        """
        if scaling=="log":
            if notes<=0: notes=1 
            # adjust nscale so smallest scale is 2 
            noctave=self._log2( ndata/largestscale/2 )
            self.nscale=notes*noctave
            self.scales=np.zeros(self.nscale,float)
            for j in range(self.nscale):
                self.scales[j] = ndata/(self.scale*(2.0**(float(self.nscale-1-j)/notes)))
        elif scaling=="linear":
            nmax=ndata/largestscale/2
            self.scales=np.arange(float(2),float(nmax))
            self.nscale=len(self.scales)
        else: raise ValueError, "scaling must be linear or log"
        return
    
    def getdata(self):
        """
        returns wavelet coefficient array
        """
        return self.cwt
    def getcoefficients(self):
        return self.cwt
    def getpower(self):
        """
        returns square of wavelet coefficient array
        """
        return (self.cwt* np.conjugate(self.cwt)).real
    def getscales(self):
        """
        returns array containing scales used in transform
        """
        return self.scales
    def getnscale(self):
        """
        return number of scales
        """
        return self.nscale

# wavelet classes    
class Morlet(Cwt):
    """
    Morlet wavelet
    """
    _omega0=6.0
    fourierwl=4* np.pi/(_omega0+ np.sqrt(2.0+_omega0**2))
    def wf(self, s_omega):
        H= np.ones(len(s_omega))
        n=len(s_omega)
        for i in range(len(s_omega)):
            if s_omega[i] < 0.0: H[i]=0.0
        # !!!! note : was s_omega/8 before 17/6/03
        xhat=0.75112554*( np.exp(-(s_omega-self._omega0)**2/2.0))*H
        return xhat

class MorletReal(Cwt):
    """
    Real Morlet wavelet
    """
    _omega0=5.0
    fourierwl=4* np.pi/(_omega0+ np.sqrt(2.0+_omega0**2))
    def wf(self, s_omega):
        H= np.ones(len(s_omega))
        n=len(s_omega)
        for i in range(len(s_omega)):
            if s_omega[i] < 0.0: H[i]=0.0
        # !!!! note : was s_omega/8 before 17/6/03
        xhat=0.75112554*( np.exp(-(s_omega-self._omega0)**2/2.0)
                         + np.exp(-(s_omega+self._omega0)**2/2.0)
                         - np.exp(-(self._omega0)**2/2.0)
                         + np.exp(-(self._omega0)**2/2.0))
        return xhat
    
class Paul4(Cwt):
    """
    Paul m=4 wavelet
    """
    fourierwl=4* np.pi/(2.*4+1.)
    def wf(self, s_omega):
        n=len(s_omega)
        xhat= np.zeros(n)
        xhat[0:n/2]=0.11268723*s_omega[0:n/2]**4* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class Paul2(Cwt):
    """
    Paul m=2 wavelet
    """
    fourierwl=4* np.pi/(2.*2+1.)
    def wf(self, s_omega):
        n=len(s_omega)
        xhat= np.zeros(n)
        xhat[0:n/2]=1.1547005*s_omega[0:n/2]**2* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class Paul(Cwt):
    """
    Paul order m wavelet
    """
    def wf(self, s_omega):
        Cwt.fourierwl=4* np.pi/(2.*self.order+1.)
        m=self.order
        n=len(s_omega)
        normfactor=float(m)
        for i in range(1,2*m):
            normfactor=normfactor*i
        normfactor=2.0**m/ np.sqrt(normfactor)
        xhat= np.zeros(n)
        xhat[0:n/2]=normfactor*s_omega[0:n/2]**m* np.exp(-s_omega[0:n/2])
        #return 0.11268723*s_omega**2*exp(-s_omega)*H
        return xhat

class MexicanHat(Cwt):
    """
    2nd Derivative Gaussian (mexican hat) wavelet
    """
    fourierwl=2.0* np.pi/ np.sqrt(2.5)
    def wf(self, s_omega):
        # should this number be 1/sqrt(3/4) (no pi)?
        #s_omega = s_omega/self.fourierwl
        #print max(s_omega)
        a=s_omega**2
        b=s_omega**2/2
        return a* np.exp(-b)/1.1529702
        #return s_omega**2*exp(-s_omega**2/2.0)/1.1529702

class DOG4(Cwt):
    """
    4th Derivative Gaussian wavelet
    see also T&C errata for - sign
    but reconstruction seems to work best with +!
    """
    fourierwl=2.0* np.pi/ np.sqrt(4.5)
    def wf(self, s_omega):
        return s_omega**4* np.exp(-s_omega**2/2.0)/3.4105319

class DOG1(Cwt):
    """
    1st Derivative Gaussian wavelet
    but reconstruction seems to work best with +!
    """
    fourierwl=2.0* np.pi/ np.sqrt(1.5)
    def wf(self, s_omega):
        dog1= np.zeros(len(s_omega),complex64)
        dog1.imag=s_omega* np.exp(-s_omega**2/2.0)/sqrt(pi)
        return dog1

class DOG(Cwt):
    """
    Derivative Gaussian wavelet of order m
    but reconstruction seems to work best with +!
    """
    def wf(self, s_omega):
        try:
            from scipy.special import gamma
        except ImportError:
            print "Requires scipy gamma function"
            raise ImportError
        Cwt.fourierwl=2* np.pi/ np.sqrt(self.order+0.5)
        m=self.order
        dog=1.0J**m*s_omega**m* np.exp(-s_omega**2/2)/ np.sqrt(gamma(self.order+0.5))
        return dog

class Haar(Cwt):
    """
    Continuous version of Haar wavelet
    """
    #    note: not orthogonal!
    #    note: s_omega/4 matches Lecroix scale defn.
    #          s_omega/2 matches orthogonal Haar
    # 2/8/05 constants adjusted to match artem eim

    fourierwl=1.0#1.83129  #2.0
    def wf(self, s_omega):
        haar= np.zeros(len(s_omega),complex64)
        om = s_omega[:]/self.currentscale
        om[0]=1.0  #prevent divide error
        #haar.imag=4.0*sin(s_omega/2)**2/om
        haar.imag=4.0* np.sin(s_omega/4)**2/om
        return haar

class HaarW(Cwt):
    """
    Continuous version of Haar wavelet (norm)
    """
    #    note: not orthogonal!
    #    note: s_omega/4 matches Lecroix scale defn.
    #          s_omega/2 matches orthogonal Haar
    # normalised to unit power

    fourierwl=1.83129*1.2  #2.0
    def wf(self, s_omega):
        haar= np.zeros(len(s_omega),complex64)
        om = s_omega[:]#/self.currentscale
        om[0]=1.0  #prevent divide error
        #haar.imag=4.0*sin(s_omega/2)**2/om
        haar.imag=4.0* np.sin(s_omega/2)**2/om
        return haar




In [5]:
animalPath = '/mnt/Data/data/Gerbils/'
localPath = '/mnt/Data/'
lfpPaths = []
for lfp in tools.locate('*.lfp',animalPath):
    lfpPaths.append(os.path.join(lfp[0],lfp[1]))
lfpPaths = sorted(lfpPaths)
lfpPaths

['/mnt/Data/data/Gerbils/G5/data/2011-08-04_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-04_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-04_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-05_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-05_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-05_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-07_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-07_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-09_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-09_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-09_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-10_sleep1_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-10_sleep2_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-10_sleep3_csc4.lfp',
 '/mnt/Data/data/Gerbils/G5/data/2011-08-14_sleep1_csc4.lfp',
 '/mnt/D

### Colors & Basics

In [6]:
colors = ["windows blue", "amber", "greyish", "faded green", "dusty purple"]
seabC = sns.xkcd_palette(colors)
seabP = sns.color_palette("Paired")
#sns.palplot(seabP)
#sns.palplot(sns.xkcd_palette(colors))
sns.set_style("ticks")
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'

## Basic lyout

In [41]:
figure = pl.figure(figsize = [15,10])
gs1 = gridspec.GridSpec(2, 1)
gs1.update(left=0.1, right=0.48,bottom=0.5, wspace=0.05)
ax0 = pl.subplot(gs1[0,:])
ax1 = pl.subplot(gs1[1, :])
ax0.xaxis.tick_bottom()
ax0.yaxis.tick_left()
ax1.xaxis.tick_bottom()
ax1.yaxis.tick_left()
gs3 = gridspec.GridSpec(3, 4)
gs3.update(left=0.06, right=0.475,top=0.39, wspace=0.05,hspace=0.01)
ax2 = pl.subplot(gs3[1:,0])
ax2up = pl.subplot(gs3[0,0])
#ax2p = ax2.twinx()
#ax2p.axes.get_yaxis().set_ticks([])
ax3 = pl.subplot(gs3[1:,1])
ax3up = pl.subplot(gs3[0,1])
#ax3p = ax3.twinx()
#ax3p.axes.get_yaxis().set_ticks([])
ax3.axes.get_yaxis().set_ticks([])
ax4 = pl.subplot(gs3[1:,2])
ax4up = pl.subplot(gs3[0,2])
#ax4p = ax4.twinx()
#ax4p.axes.get_yaxis().set_ticks([])
ax4.axes.get_yaxis().set_ticks([])
ax5 = pl.subplot(gs3[1:,3])
ax5up = pl.subplot(gs3[0,3])
#ax5p = ax5.twinx()
#ax5p.axes.get_yaxis().set_ticks([])
ax5.axes.get_yaxis().set_ticks([])
#ax6 = pl.subplot(gs1[-1,0])
#ax7 = pl.subplot(gs1[-1,1])
#ax8 = pl.subplot(gs1[-1,2])
#ax9 = pl.subplot(gs1[-1,3])
axarr = [ax1,ax2,ax3,ax4,ax5]
gs1.tight_layout(figure,rect=[0, 0.5, 0.5, 1])
for axxx in axarr:
    axxx.xaxis.tick_bottom()
    axxx.yaxis.tick_left()
#gs3.tight_layout(figure,rect=[0.0, 0, 0.5, 0.33])

## Sleep phases

In [8]:
SleepEpochs = pd.read_pickle(animalPath+'SleepEpochs.pd')
df = pd.read_pickle(animalPath+'psdData.pd')

In [9]:
remEP = SleepEpochs[SleepEpochs.epoch=='REM']
swsEP = SleepEpochs[SleepEpochs.epoch=='SWS']
print (remEP.t1-remEP.t0).mean(), (remEP.t1-remEP.t0).min(), (remEP.t1-remEP.t0).max()
print (swsEP.t1-swsEP.t0).mean(),(swsEP.t1-swsEP.t0).min(),(swsEP.t1-swsEP.t0).max()

113.170212766 25 405
107.833333333 20 400


In [10]:
lfp = pkl.load(open('/mnt/Data/data/Gerbils/G5/data/2011-08-07_sleep2_csc4.lfp','rb'))

In [17]:
traj = None
HDtraj = None
def getTraj(trajFile):
    global traj
    global HDtraj
    print 'loading', trajFile
    traj = trajectory.load_nvtFile(trajFile, 'linearMaze', showHeader=False)
    HDtraj = traj[1]        # head direction
    traj = traj[0]          # trajectory
getTraj('/mnt/Data/data/Gerbils/G5/data/2011-08-07/sleep2/VT1.nvt')

loading /mnt/Data/data/Gerbils/G5/data/2011-08-07/sleep2/VT1.nvt


IOError: [Errno 2] No such file or directory: '/mnt/Data/data/Gerbils/G5/data/2011-08-07/sleep2/VT1.nvt'

In [18]:
traj.times[-1]-traj.times[0],lfp.duration()/1e3

AttributeError: 'NoneType' object has no attribute 'times'

In [19]:
Pxx.shape

NameError: name 'Pxx' is not defined

In [11]:
Pxx, freqs, t = lfp.spectrogram(minFreq=.1,maxFreq=55,overlap=1024,windowSize=2048,whiten=True)
BandAvg = []
for col in range(Pxx.shape[1]):
    Pxx[:,col] = np.convolve(Pxx[:,col],scsig.gaussian(10,3),'same')
    BandAvg.append(np.array([np.average(Pxx[:,col][0:15]),np.average(Pxx[:,col][16:31]),np.average(Pxx[:,col][32:])]))
for row in range(Pxx.shape[0]):
    Pxx[row,:] = np.convolve(Pxx[row,:],scsig.gaussian(20,5),'same')
Pxx = np.clip(Pxx,0,Pxx.mean()+1.4*Pxx.std())


AttributeError: 'list' object has no attribute 'spectrogram'

In [21]:
#im = ax0.pcolormesh(t/1000.0,freqs,Pxx,cmap=mycmps.viridis)
im = ax0.imshow(Pxx,aspect='auto',cmap='viridis')
ax0.axvspan(168,730,color='w',alpha=0.3)
#ax0.axvspan(790,900,color=seabC[1],alpha=0.3)
#ax0.axvspan(1430,1610,color=seabC[1],alpha=0.3)
#ax0.axhline(5,c='w',lw=0.5)
ax0.set_xlim(0,1000)
#ax0.set_xticks([0,200,400,600,800,1000])
#ax0.set_xticklabels(['0','100','200','300','400','500'])

(0, 1000)

In [22]:
ax0.cla()
ax1.cla()

In [16]:
sfig,axx = pl.subplots(1,1)
axx.imshow(pwr,aspect='auto')

NameError: name 'pwr' is not defined

In [10]:
#ax1.set_yscale('log')
colList = [seabP[0],seabP[2]]
decIdx = df.index[::35]
dfsub = df.loc[decIdx]
dfbelow = dfsub[dfsub.f < 56.5]
dfabove = dfsub[dfsub.f > 62.5]
#sns.tsplot(value='psd',time='f',condition='epoch',unit='epochNo',data=dfbelow,ax=ax1,color=colList)
sns.tsplot(value='psd',time='f',condition='epoch',unit='epochNo',data=df,ax=ax1,color=colList,legend=False)


In [29]:
ax1.cla()

In [42]:
ax0.set_yticks([10,20,30,40,50,159])
ax0.set_yticklabels([r'4',r'8',r'12',r'16',r'20'],fontsize=14)
ax0.set_xlabel('Time (s)',fontsize=14)
ax0.set_xticklabels([r'0',r'100',r'200',r'300',r'400',r'500',r'600'],fontsize=14)
ax0.set_ylabel('Frequency (Hz)',fontsize=14)
ax0.set_ylim(0.3,170)
ax1.set_xlabel('Frequency (Hz)',fontsize=14)
#ax1.set_xticklabels([0,50,100,150,200,250],fontsize=14)
ax1.set_ylabel('Power (dB)',fontsize=14)
ax1.set_xticklabels([r'0',r'50',r'100',r'150',r'200',r'250'],fontsize=14)
ax1.set_yticks([])
ax1.set_yticklabels([])
ax1.set_xlim([0,250])
ax1.set_ylim([5e7,3.1e8])
ax1.legend(loc=4)
#ax2.set_axis_off()
gs1.tight_layout(figure,rect=[0, 0.4, 0.5, 1])

## Ripple Examples

In [12]:
rippDB = pd.read_pickle(animalPath+'RipplesTimeDB.pd')
c  = np.int16(np.loadtxt('/mnt/Data/data/Gerbils/pink.clu.1'))[1:]
wc = np.int16(np.loadtxt('/mnt/Data/data/Gerbils/white.clu.1'))[1:]
np.place(wc,wc==3,[1])
np.place(c,c==2,[1])
np.place(c,c==3,[2])
rippDB['wc'] = wc
rippDB['c'] = c

In [15]:
print selS[selS.epoch=='rem'].shape,selG[selG.epoch=='rem'].shape
print np.unique(rippDB.date)
print len(lfpPaths)
print rippDB.shape
print selS.shape,selG.shape

(470, 24) (5201, 24)
['2011-08-05' '2011-08-07' '2011-08-09' '2011-08-10' '2011-09-19'
 '2011-09-23']
26
(14481, 24)
(2339, 24) (10674, 24)


In [14]:
swrEx = np.logical_and(np.logical_and(rippDB.HilbertAbsPeak>-5.9,rippDB.c==1),rippDB.wc==1).as_matrix()
gbsEx = np.logical_and(np.logical_and(rippDB.HilbertAbsPeak>-8.9,rippDB.c==2),rippDB.wc==2).as_matrix()
selS = rippDB.loc[swrEx]
selG = rippDB.loc[gbsEx]

In [100]:
fiz = pl.figure(figsize=[4,6])
bx = pl.Axes(fiz, [0., 0., 1., 1.])
bx.set_axis_off()
fiz.add_axes(bx)
cxx = bx.twinx()
pl.axis('off')
wavelet=Morlet
maxscale=3.05
notes=64
scaling="log" #or "linear"
#scaling="linear"
plotpower2d=True
# set up some data
Ns=2000
for kk,item in enumerate(selS[selS.epoch=='rem'].index):
    if kk>-460:
        for path in lfpPaths:
            session = selS.session[item].split('0')[0]+selS.session[item].split('0')[1]
            if path.find(selS.date[item])>0 and path.find(session)>0:
                lfp = pkl.load(open(path,'rb'))
                tP = selS['t_peak'][item]
                t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
                t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
                sig = lfp.signal[t1:t2]
                wsig = lfp.signal_white[t1:t2]
                cw=wavelet(wsig,maxscale,notes,scaling=scaling)
                scales=cw.getscales()     
                cwt=cw.getdata()
                # power spectrum
                pwr=cw.getpower()
                scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
                # scales
                y=cw.fourierwl*scales
                for jj in range(pwr.shape[1]):
                    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(15,15),'same')
                bx.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = mycmps.magma)
                bx.axhline(150,linewidth='1.5',color = 'w',linestyle='--')
                bx.set_xlim(-0.3,0.3)
                bx.set_ylim(4,250)
                #bx.set_axis_off()
                cxx.set_aspect('auto')
                #ax4up.plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=0.9,alpha=0.000085)
                cxx.plot(np.linspace(-0.512,0.512,1024),sig,c='w',lw=1.0)
                cxx.set_xlim(-.3,0.3)
                cxx.set_ylim(1.5*sig.min(),2*sig.max())
                #pl.axis('off')
                fiz.savefig('/mnt/Data/ownCloud/Plots/Gerbil-plots/wvSample-'+str(item)+'.png',transparent=True,dpi=200)
                cxx.cla()
                bx.cla()

In [95]:
1 in [1,2,3]

True

In [16]:
fiz = pl.figure(figsize=[4,6])
bx = pl.Axes(fiz, [0., 0., 1., 1.])
bx.set_axis_off()
fiz.add_axes(bx)
cxx = bx.twinx()
pl.axis('off')
wavelet=Morlet
maxscale=3.05
notes=64
scaling="log" #or "linear"
#scaling="linear"
plotpower2d=True
# set up some data
Ns=2000
for kk,item in enumerate(selG.index):
    if kk==31:
        for path in lfpPaths:
            session = selG.session[item].split('0')[0]+selG.session[item].split('0')[1]
            if path.find(selG.date[item])>0 and path.find(session)>0:
                lfp = pkl.load(open(path,'rb'))
                tP = selG['t_peak'][item]
                t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
                t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
                sig = lfp.signal[t1:t2]
                wsig = lfp.signal_white[t1:t2]
                cw=wavelet(wsig,maxscale,notes,scaling=scaling)
                scales=cw.getscales()     
                cwt=cw.getdata()
                # power spectrum
                pwr=cw.getpower()
                scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
                # scales
                y=cw.fourierwl*scales
                for jj in range(pwr.shape[1]):
                    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(15,15),'same')
                bx.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = mycmps.magma)
                bx.axhline(150,linewidth='1.5',color = 'w',linestyle='--')
                bx.set_aspect('auto')
                bx.set_xlim(-0.3,0.3)
                bx.set_ylim(4,250)
                #ax2up.axhline(sig.min(),alpha=0.001)
                #ax2up.plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=0.9,alpha=0.000085)
                ax4up.plot(np.linspace(-0.512,0.512,1024),sig+45000,c='k',lw=1.0,alpha=0.85)
                ax4up.set_xlim(-0.3,0.3)
                fiz.savefig('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample31.png',transparent=True,dpi=600)
                #cxx.cla()
                bx.cla()
    if kk==81:
        for path in lfpPaths:
            session = selG.session[item].split('0')[0]+selG.session[item].split('0')[1]
            if path.find(selG.date[item])>0 and path.find(session)>0:
                lfp = pkl.load(open(path,'rb'))
                tP = selG['t_peak'][item]
                t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
                t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
                sig = lfp.signal[t1:t2]
                wsig = lfp.signal_white[t1:t2]
                cw=wavelet(wsig,maxscale,notes,scaling=scaling)
                scales=cw.getscales()     
                cwt=cw.getdata()
                # power spectrum
                pwr=cw.getpower()
                scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
                # scales
                y=cw.fourierwl*scales
                for jj in range(pwr.shape[1]):
                    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(15,15),'same')
                bx.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = mycmps.magma)
                bx.axhline(150,linewidth='1.5',color = 'w',linestyle='--')
                bx.set_aspect('auto')
                bx.set_xlim(-0.3,0.3)
                bx.set_ylim(4,250)
                #ax3up.axhline(sig.min(),alpha=0.001)
                #ax3up.plot(np.linspace(-0.512,0.512,1024),sig,c='w',lw=0.9,alpha=0.000085)
                ax5up.plot(np.linspace(-0.512,0.512,1024),sig+45000,c='k',lw=1.0,alpha=0.85)
                ax5up.set_xlim(-0.3,0.3)
                fiz.savefig('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample81.png',transparent=True,dpi=600)
                #cxx.cla()
                bx.cla()
for kk,item in enumerate(selS.index):
    if kk==51:
        for path in lfpPaths:
            session = selS.session[item].split('0')[0]+selS.session[item].split('0')[1]
            if path.find(selS.date[item])>0 and path.find(session)>0:
                lfp = pkl.load(open(path,'rb'))
                tP = selS['t_peak'][item]
                t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
                t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
                sig = lfp.signal[t1:t2]
                wsig = lfp.signal_white[t1:t2]
                cw=wavelet(wsig,maxscale,notes,scaling=scaling)
                scales=cw.getscales()     
                cwt=cw.getdata()
                # power spectrum
                pwr=cw.getpower()
                scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
                # scales
                y=cw.fourierwl*scales
                for jj in range(pwr.shape[1]):
                    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(15,15),'same')
                bx.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = mycmps.magma)
                bx.axhline(150,linewidth='1.5',color = 'w',linestyle='--')
                bx.set_xlim(-0.3,0.3)
                bx.set_ylim(4,250)
                #bx.set_axis_off()
                ax2up.set_aspect('auto')
                #ax4up.plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=0.9,alpha=0.000085)
                ax2up.plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=1.0)
                ax2up.set_xlim(-0.3,0.3)
                #pl.axis('off')
                fiz.savefig('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample51.png',transparent=True,dpi=600)
                #cxx.cla()
                bx.cla()
    if kk==33:
        for path in lfpPaths:
            session = selS.session[item].split('0')[0]+selS.session[item].split('0')[1]
            if path.find(selS.date[item])>0 and path.find(session)>0:
                lfp = pkl.load(open(path,'rb'))
                tP = selS['t_peak'][item]
                t1 = tools.findNearest(lfp.timeAxis,tP-256)[0]
                t2 = tools.findNearest(lfp.timeAxis,tP+256)[0]
                sig = lfp.signal[t1:t2]
                wsig = lfp.signal_white[t1:t2]
                cw=wavelet(wsig,maxscale,notes,scaling=scaling)
                scales=cw.getscales()     
                cwt=cw.getdata()
                # power spectrum
                pwr=cw.getpower()
                scalespec=np.sum(pwr,axis=1)/scales # calculate scale spectrum
                # scales
                y=cw.fourierwl*scales
                for jj in range(pwr.shape[1]):
                    pwr[:,jj] = np.convolve(pwr[:,jj],scsig.gaussian(15,15),'same')
                bx.pcolormesh(np.linspace(-0.512,0.512,1024),y[::-1],(pwr),cmap = mycmps.magma)
                #bx.imshow(pwr,aspect='auto',cmap = 'magma')
                bx.axhline(150,linewidth='1.5',color = 'w',linestyle='--')
                bx.set_xlim(-0.3,0.3)
                bx.set_ylim(4,250)
                #bx.set_axis_off()
                ax3up.set_aspect('auto')
                #cxx.axhline(sig.min(),alpha=0.001)
                #cxx.plot(np.linspace(-0.512,0.512,1024),sig,c='w',lw=0.9,alpha=0.000085)
                ax3up.plot(np.linspace(-0.512,0.512,1024),sig,c='k',lw=1.0,)
                ax3up.set_xlim(-0.3,0.3)
                #pl.axis('off')
                fiz.savefig('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample33.png',transparent=True,dpi=600)
                #cxx.cla()
                bx.cla()

IOError: [Errno 2] No such file or directory: '/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample31.png'

In [17]:
im = pl.imread('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample31.png')
ax4.imshow(im,aspect='auto',cmap = 'magma')
im = pl.imread('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample81.png')
ax5.imshow(im,aspect='auto',cmap = 'magma')
im = pl.imread('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample51.png')
ax2.imshow(im,aspect='auto',cmap = 'magma')
im = pl.imread('/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample33.png')
ax3.imshow(im,aspect='auto',cmap = 'magma')

IOError: [Errno 2] No such file or directory: '/home/chenani/ownCloud/Plots/Gerbil-plots/wvSample31.png'

In [18]:
ax2.set_xticks([0,1200,2400])
ax2up.set_xticks([])
ax2up.set_yticks([])
#ax2up.set_axis_off()
ax2.set_xticklabels([r'$-0.3$',r'$0$',r'$+0.3$'],fontsize=14)
ax2.set_xlabel('Time (s)',fontsize=14)
ax2.set_ylabel('Frequency (Hz)',fontsize=14)
#ax2up.set_ylabel('Voltage (uV)',fontsize=14)
ax2.set_yticks([600,1450,2200,3000])#,fontsize=14)
ax2.set_yticklabels([r'200',r'150',r'100',r'50'],fontsize=14)
#ax2p.set_axis_off()
ax3.set_axis_off()
#ax3up.set_axis_off()
ax3up.set_xticks([])
ax3up.set_yticks([])
#ax3p.set_axis_off()
#ax4.set_axis_off()
ax4up.set_xticks([])
ax4up.set_yticks([])
#ax4up.set_axis_off()
ax5.set_axis_off()
#ax5up.set_axis_off()
ax5up.set_xticks([])
ax5up.set_yticks([])

[]

In [116]:
ax4.cla()
ax4up.cla()
ax5.cla()
ax5up.cla()
ax2.cla()
ax2up.cla()
ax3.cla()
ax3up.cla()
#ax3p.cla()
#ax4p.cla()
#ax5p.cla()
#ax1.cla()
#ax16.cla()

In [43]:
gs2 = gridspec.GridSpec(12,12)
gs2.update(left=0.55, right=0.98, hspace=0.05)
ax10 = pl.subplot(gs2[:4, :6])
ax11 = pl.subplot(gs2[:4, 6:])
ax12 = pl.subplot(gs2[8:,:6])
ax13 = pl.subplot(gs2[4:8,:6])
ax14 = pl.subplot(gs2[4:8,6:])
ax16 = pl.subplot(gs2[8:,6:])
gs2.tight_layout(figure,rect=[0.51, 0.01, 0.99, 1])
for axxx in [ax10,ax11,ax12,ax13,ax14,ax16,]:
    axxx.xaxis.tick_bottom()
    axxx.yaxis.tick_left()

In [79]:
rippDB = pd.read_pickle(animalPath+'RipplesTimeDB.pd')
rippPSD = pkl.load(open(animalPath+'rippPSD.npArr','rb'))
whiteRippPSD = pkl.load(open(animalPath+'whiteRippPSD.npArr','rb'))
SWRsignal = pkl.load(open(animalPath+'rippSignal.npArr','rb'))
ss = []
for iii,item in enumerate(SWRsignal):
    if item.size < 200:
        SWRsignal[iii] = np.append(item,np.zeros(200 - item.size))
        print iii
    ss.append(item)
ss = np.array(ss)

In [80]:
rippDB = rippDB[rippDB.epoch=='sws']
rippPSD= rippPSD[rippDB.index,:]
ss = ss[rippDB.index,:]

In [81]:
Rcond = np.logical_and(rippDB.c ==1, rippDB.wc == 1)
Gcond = np.logical_and(rippDB.c ==2, rippDB.wc == 2)
cond12 = np.logical_and(rippDB.c ==1, rippDB.wc == 2)
cond21 = np.logical_and(rippDB.c ==2, rippDB.wc == 1)
p = np.array([0.0, 7.8125, 15.625, 23.4375, 31.25, 39.0625, 46.875, 54.6875, 62.5, 70.3125, 78.125,
 85.9375, 93.75, 101.5625, 109.375, 117.1875, 125.0, 132.8125, 140.625, 148.4375, 156.25, 164.0625,
 171.875, 179.6875, 187.5, 195.3125, 203.125, 210.9375, 218.75, 226.5625, 234.375, 242.1875, 250.0,
 257.8125, 265.625, 273.4375, 281.25, 289.0625, 296.875, 304.6875, 312.5, 320.3125, 328.125, 335.9375,
 343.75, 351.5625, 359.375, 367.1875, 375.0, 382.8125, 390.625, 398.4375, 406.25, 414.0625, 421.875,
 429.6875, 437.5, 445.3125, 453.125, 460.9375, 468.75, 476.5625, 484.375, 492.1875, 500.0, 507.8125,
 515.625, 523.4375, 531.25, 539.0625, 546.875, 554.6875, 562.5, 570.3125, 578.125, 585.9375, 593.75,
 601.5625, 609.375, 617.1875, 625.0, 632.8125, 640.625, 648.4375, 656.25, 664.0625, 671.875, 679.6875,
 687.5, 695.3125, 703.125, 710.9375, 718.75, 726.5625, 734.375, 742.1875, 750.0, 757.8125, 765.625,
 773.4375, 781.25, 789.0625, 796.875, 804.6875, 812.5, 820.3125, 828.125, 835.9375, 843.75, 851.5625,
 859.375, 867.1875, 875.0, 882.8125, 890.625, 898.4375, 906.25, 914.0625, 921.875, 929.6875, 937.5,
 945.3125, 953.125, 960.9375, 968.75, 976.5625, 984.375, 992.1875, 1000.0])

In [78]:
sns.tsplot(rippPSD[Rcond.as_matrix(),:],time = p[:35],ax=ax10,color='#a6a5d3')
sns.tsplot(rippPSD[Rcond.as_matrix(),:],time = p[:35],err_style="ci_bars", interpolate=False,ax=ax10,condition='SWR',color='#a6a5d3')
sns.tsplot(rippPSD[Gcond.as_matrix(),:],time = p[:35],ax=ax10,color='#f0b25c')
sns.tsplot(rippPSD[Gcond.as_matrix(),:],time = p[:35],err_style="ci_bars", interpolate=False,ax=ax10,condition='FGB',color='#f0b25c')
sns.tsplot(ss[Rcond.as_matrix(),:],time = np.arange(-50,50,0.5),c='#a6a5d3',ax=ax11,condition='SWR')
sns.tsplot(ss[Gcond.as_matrix(),:],time = np.arange(-50,50,0.5),c='#f0b25c',ax=ax11,condition='FGB')

In [82]:
print Rcond.sum(),Gcond.sum(),cond12.sum(),cond21.sum()

1371 3073 450 13


In [58]:
ax10.set_xlabel('Frequency (Hz)',fontsize=14)
ax10.set_xticks([50,150,250])
ax10.set_xticklabels([r'50',r'150',r'250'],fontsize=14)
ax10.set_ylabel('PSD (a.u.)',fontsize=14)
ax10.set_yticks([])
ax11.set_ylabel('Voltage (uV)',fontsize=14)
ax11.set_xlabel('Time (ms)',fontsize=14)
ax11.set_xticks([-40,-20,0,20,40])
ax11.set_xticklabels([r'-40',r'-20',r'0',r'20',r'40'],fontsize=14)
ax11.set_ylim(-1800,1800)
ax11.set_yticks([-1500,-500,500,1500])
ax11.set_yticklabels([r'-1500',r'-500',r'500',r'1500'],fontsize=14)


In [62]:
ax10.cla()
ax11.cla()
ax14.cla()

## DistPlots

In [74]:
eventID = np.array(rippDB.shape[0]*['NA'])
for iii,item in enumerate(Rcond.as_matrix()):
    if item:
        eventID[iii] = 'SWR'
    if Gcond.as_matrix()[iii]:
        eventID[iii] = 'FGB'
rippDB['eventID'] = eventID
rippDB['duration'] = (rippDB.t_f - rippDB.t_i)
###Calculating Intervals
iri = np.array(eventID.size*[np.nan])
igi = np.array(eventID.size*[np.nan])
Ridx = rippDB[Rcond].index
Gidx = rippDB[Gcond].index
for kk,idx in enumerate(Ridx):
    if rippDB.session[Ridx[kk]]== rippDB.session[Ridx[kk-1]]:
        iri[kk] = rippDB.loc[Ridx[kk]].t_peak - rippDB.loc[Ridx[kk-1]].t_peak
for kk,idx in enumerate(Gidx):
    if rippDB.session[Gidx[kk]]== rippDB.session[Gidx[kk-1]]:
        igi[kk] = rippDB.t_peak[Gidx[kk]] - rippDB.t_peak[Gidx[kk-1]]


In [73]:
iri

array([    nan,    88.5,  6228.5, ...,     nan,     nan,     nan])

In [75]:
Ridx.size,Gidx.size

(1371, 3073)

In [76]:
ax13.set_yscale('log')
#ax13.set_xscale('log')
w = np.logical_not(np.isnan(iri))
u = np.logical_not(np.isnan(igi))
histiri,bins = np.histogram(iri[w],bins=np.arange(0,20000,50),normed=True)
histigiAvg = np.zeros(bins.size-1)
for i in range(5):
    histigi,bins = np.histogram(np.random.choice(igi[u],iri[w].size),bins=np.arange(0,20000,50),normed=True)
    histigiAvg += histigi
histigiAvg = histigiAvg/2.0
#histiei,bins = np.histogram(rippDB.iei,bins=bins,normed=True)
ax13.scatter(bins[:-1],histiri,c='#a6a5d3',label='SWR')
ax13.scatter(bins[:-1],histigiAvg,c='#f0b25c',label='FGB')
ax13.set_xlabel('IEI (ms)',fontsize=14)
ax13.set_ylabel('Normalized count',fontsize=14)
ax13.legend()
ax13.set_ylim(1e-6,0.01)
ax13.set_xlim(-1000,10000)
ax13.set_yticks([1e-5,1e-3])
ax13.set_yticklabels([r'$10^{-5}$',r'$10^{-3}$'],fontsize=14)
ax13.set_xticks([1e2,1e4])
ax13.set_xticklabels([r'$10^2$',r'$10^4$'],fontsize=14)

In [23]:
histigi.sum(),histiri.sum()

(0.020000000000000004, 0.019999999999999997)

In [24]:
ax13.cla()

In [77]:
ax14.set_yscale('log')
ax14.set_xscale('log')
dBins = np.arange(50,5000,10)
RDhist,bins = np.histogram(rippDB.duration[Rcond],dBins,normed=True)
GDhist,bins = np.histogram(rippDB.duration[Gcond],dBins,normed=True)
ax14.scatter(bins[:-1],RDhist,c='#a6a5d3',label='SWR')
ax14.scatter(bins[:-1],GDhist,c='#f0b25c',label='FGB')
ax14.legend()
ax14.set_xlabel('Duration (ms)',fontsize=14)
ax14.set_ylabel('Normalized count',fontsize=14)
ax14.set_ylim(1e-5,1e-1)
ax14.set_xlim(20,800)
ax14.set_yticks([1e-4,1e-2])
ax14.set_yticklabels([r'$10^{-4}$',r'$10^{-2}$'],fontsize=14)
ax14.set_xticks([5e1,2e2,6e2])
ax14.set_xticklabels([r'$50$',r'$200$',r'$600$'],fontsize=14)

In [135]:
ax14.cla()

## Barplots

In [37]:
set(rippDB.session)

{'sleep01', 'sleep02'}

In [38]:
rippDB.replace('sleep03','sleep02',inplace=True)

In [56]:
Rsws = rippDB[Rcond]['c']==1
Rrem = rippDB[Rcond]['c']==1
Gsws = rippDB[Gcond]['c']==2
Grem = rippDB[Gcond]['c']==2
RswsIEI = np.array([])
RremIEI = np.array([])
GswsIEI = np.array([])
GremIEI = np.array([])
for kk,idx in enumerate(rippDB[Rcond][Rsws].index):
    prev = rippDB[Rcond][Rsws].index[kk-1]
    if rippDB.session[idx] == rippDB.session[prev]:
        RswsIEI = np.append(RswsIEI,rippDB.t_peak[idx]-rippDB.t_peak[prev])
for kk,idx in enumerate(rippDB[Rcond][Rrem].index):
    prev = rippDB[Rcond][Rrem].index[kk-1]
    if rippDB.session[idx] == rippDB.session[prev]:
        RremIEI = np.append(RremIEI,rippDB.t_peak[idx]-rippDB.t_peak[prev])
for kk,idx in enumerate(rippDB[Gcond][Gsws].index):
    prev = rippDB[Gcond][Gsws].index[kk-1]
    if rippDB.session[idx] == rippDB.session[prev]:
        GswsIEI = np.append(GswsIEI,rippDB.t_peak[idx]-rippDB.t_peak[prev])
for kk,idx in enumerate(rippDB[Gcond][Grem].index):
    prev = rippDB[Gcond][Grem].index[kk-1]
    if rippDB.session[idx] == rippDB.session[prev]:
        GremIEI = np.append(GremIEI,rippDB.t_peak[idx]-rippDB.t_peak[prev])

IEImeans = []
IEIstds = []
IEIsems = []
for item in [RswsIEI,RremIEI,GswsIEI,GremIEI]:
    validx = np.logical_and(item>0,item<1e4)
    IEImeans.append(item[validx].mean())
    IEIstds.append(item[validx].std())
    IEIsems.append(sem(item[validx]))
#############################################
#############################################
#############################################
Dmeans = [rippDB.duration[Rcond].mean(),
          rippDB.duration[Rcond].mean(),
          rippDB.duration[Gcond].mean(),
          rippDB.duration[Gcond].mean()]
Dstds = [ rippDB.duration[Rcond].std(),
          rippDB.duration[Rcond].std(),
          rippDB.duration[Gcond].std(),
          rippDB.duration[Gcond].std()]
Dsems = [sem(rippDB.duration[Rcond]),
          sem(rippDB.duration[Rcond]),
          sem(rippDB.duration[Gcond]),
          sem(rippDB.duration[Gcond])]

In [57]:
#f,ax12 = pl.subplots(1,1)
ax12.axvspan(0.02,0.1,ymin=0,ymax=IEImeans[0]/5000,color= '#a6a5d3',label='SWS-SWR')
ax12.axvline(.06,ymin=(IEImeans[0]-IEIsems[0])/5e3,ymax=(IEImeans[0]+IEIsems[0])/5e3,linewidth=3,color='k')
#ax12.axhline(((IEImeans[0]+IEIsems[0])/5e3),xmin=0.04,xmax=0.08)
#################################
ax12.axvspan(0.1,0.18,ymin=0,ymax=IEImeans[1]/5000,color= '#a6a5d3',alpha=0.5,label='REM-SWR')
ax12.axvline(.14,ymin=(IEImeans[1]-IEIsems[1])/5000,ymax=(IEImeans[1]+IEIsems[1])/5000,linewidth=3,color='k')
#ax12.axhline((IEImeans[1]+IEIsems[1])/5000,xmin=0.12,xmax=0.16)
#################################
ax12.axvspan(0.22,0.30,ymin=0,ymax=IEImeans[2]/5000,color= '#f0b25c',label='SWS-FGB')
ax12.axvline(.26,ymin=(IEImeans[2]-IEIsems[2])/5000,ymax=(IEImeans[2]+IEIsems[2])/5000,linewidth=3,color='k')
#ax12.axhline((IEImeans[2]+IEIsems[2])/5000,xmin=0.24,xmax=0.28)
#################################
ax12.axvspan(0.30,0.38,ymin=0,ymax=IEImeans[3]/5000,color= '#f0b25c',alpha=0.5,label='REM-FGB')
ax12.axvline(.34,ymin=(IEImeans[3] - IEIsems[3])/5000,ymax=(IEImeans[3]+IEIsems[3])/5000,linewidth=3,color='k')
#ax12.axhline((IEImeans[3]+IEIsems[3])/5000,xmin=0.32,xmax=0.36)
#################################
ax12.set_yticks([0.25,0.50,0.75])
ax12.set_yticklabels([r'$1250$',r'$2500$',r'$3750$'],fontsize=14)
ax12.set_xticklabels(['IEI','Duration'])
ax12.set_ylabel('IEI (ms)',fontsize=14)
######################
ax12p = ax12.twinx()
ax12p.axvspan(0.6,0.68,ymin=0,ymax=Dmeans[0]/150,color= '#a6a5d3')
ax12p.axvline(.64,ymin=(Dmeans[0]-Dsems[0])/150,ymax=(Dmeans[0]+Dsems[0])/150,linewidth=3,color='k')
#ax12p.axhline((Dmeans[0]+Dstds[0])/150,xmin=0.52,xmax=0.56)
##################################
ax12p.axvspan(0.68,0.76,ymin=0,ymax=Dmeans[1]/150,color= '#a6a5d3',alpha=0.5)
ax12p.axvline(.72,ymin=(Dmeans[1]-Dsems[1])/150,ymax=(Dmeans[1]+Dsems[1])/150,linewidth=3,color='k')
#ax12p.axhline((Dmeans[1]+Dstds[1])/150,xmin=0.6,xmax=0.64)
##################################
ax12p.axvspan(0.8,0.88,ymin=0,ymax=Dmeans[2]/150,color= '#f0b25c')
ax12p.axvline(.84,ymin=(Dmeans[2]-Dsems[2])/150,ymax=(Dmeans[2]+Dsems[2])/150,linewidth=3,color='k')
#ax12p.axhline((Dmeans[2]+Dstds[2])/150,xmin=0.72,xmax=0.76)
##################################
ax12p.axvspan(0.88,0.96,ymin=0,ymax=Dmeans[3]/150,color= '#f0b25c',alpha=0.5)
ax12p.axvline(.92,ymin=(Dmeans[3]-Dsems[3])/150,ymax=(Dmeans[3]+Dsems[3])/150,linewidth=3,color='k')
#ax12p.axhline((Dmeans[3]+Dstds[3])/150,xmin=0.78,xmax=0.82)

ax12p.set_xlim(0,1)
ax12p.set_yticks([0.2,0.4,0.6,0.8])
ax12p.set_yticklabels([r'$30$',r'$60$',r'$90$',r'$120$'],fontsize=14)
ax12p.set_xticks([0.225,0.725])
ax12p.set_ylabel('Duration (ms)',fontsize=14)
ax12.legend()
#ax12.set_axis_off()

In [55]:
ax12.cla()
ax12p.cla()

## Population Rate plot

In [58]:
rr = pkl.load(open('/mnt/Data/data/Gerbils/popRateRipp.npArr','rb'))
gg = pkl.load(open('/mnt/Data/data/Gerbils/popRateFGB.npArr','rb'))

In [59]:
sns.tsplot(rr,ax=ax16,color='#a6a5d3')
sns.tsplot(rr,ax=ax16,color='#a6a5d3',err_style="ci_bars", interpolate=False,condition='SWR')
sns.tsplot(gg,ax=ax16,color='#f0b25c')
sns.tsplot(gg,ax=ax16,color='#f0b25c',err_style="ci_bars", interpolate=False,condition='FGB')
ax16.set_xticks([-0,4.5,9])
ax16.set_xticklabels([r'$-50$',r'$0$',r'$+50$'],fontsize=14)
ax16.set_yticks([-0,0.8,1.6])
ax16.set_yticklabels([r'$-0$',r'$0.8$',r'$1.6$'],fontsize=14)
ax16.set_ylim(0,1.7)
ax16.set_xlabel('Time (ms)',fontsize=14)
ax16.set_ylabel(r'Population rate (S.D.)',fontsize=14)
gs2.tight_layout(figure,rect=[0.52, 0.055, 0.99, 0.98])

In [156]:
ax16.cla()

In [147]:
lots/Gerbil-plots/gFig-PNG-background.pdf')

In [60]:
from scipy.stats import ranksums

In [61]:
rsTest = ranksums(RswsIEI,RremIEI)
print 'SWS-SWR vs. REM-SWR IEI p_value = %s' %rsTest.pvalue
rsTest = ranksums(RswsIEI,GswsIEI)
print 'SWS-SWR vs. SWS-FGB IEI p_value = %s' %rsTest.pvalue
rsTest = ranksums(GremIEI,GswsIEI)
print 'SWS-FGB vs. REM-FGB IEI p_value = %s' %rsTest.pvalue
rsTest = ranksums(RremIEI,GremIEI)
print 'REM-SWR vs. REM-FGB IEI p_value = %s' %rsTest.pvalue

SWS-SWR vs. REM-SWR IEI p_value = 1.0
SWS-SWR vs. SWS-FGB IEI p_value = 1.1944043036e-280
SWS-FGB vs. REM-FGB IEI p_value = 1.0
REM-SWR vs. REM-FGB IEI p_value = 1.1944043036e-280


In [62]:
RswsD = rippDB.duration[Rcond]
RremD = rippDB.duration[Rcond]
GswsD = rippDB.duration[Gcond]
GremD = rippDB.duration[Gcond]

In [63]:
rsTest = ranksums(RswsD,RremD)
print 'SWS-SWR vs. REM-SWR Duration p_value = %s' %rsTest.pvalue
rsTest = ranksums(RswsD,GswsD)
print 'SWS-SWR vs. SWS-FGB Duration p_value = %s' %rsTest.pvalue
rsTest = ranksums(GremD,GswsD)
print 'SWS-FGB vs. REM-FGB Duration p_value = %s' %rsTest.pvalue
rsTest = ranksums(RremD,GremD)
print 'REM-SWR vs. REM-FGB Duration p_value = %s' %rsTest.pvalue

SWS-SWR vs. REM-SWR Duration p_value = 1.0
SWS-SWR vs. SWS-FGB Duration p_value = 2.8269332562e-07
SWS-FGB vs. REM-FGB Duration p_value = 1.0
REM-SWR vs. REM-FGB Duration p_value = 2.8269332562e-07
